## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-30-15-45-36 +0000,nypost,Blood pressure medicine recalled over high lev...,https://nypost.com/2025/10/30/business/blood-p...
1,2025-10-30-15-43-33 +0000,bbc,Starmer rules out investigation after Reeves a...,https://www.bbc.com/news/articles/cd04d0yxnrvo...
2,2025-10-30-15-41-58 +0000,nyt,Milei’s Win in Argentina Signals Rebuke of the...,https://www.nytimes.com/2025/10/27/world/ameri...
3,2025-10-30-15-41-23 +0000,nyt,Study Finds Evidence That Text-Based Therapy E...,https://www.nytimes.com/2025/10/30/health/text...
4,2025-10-30-15-40-59 +0000,nypost,Katie Miller threatens to storm off Piers Morg...,https://nypost.com/2025/10/30/media/katie-mill...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2365/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,59
148,new,30
116,china,21
71,will,16
84,xi,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
20,2025-10-30-15-17-58 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...,153
226,2025-10-30-02-28-18 +0000,nypost,Trump says ‘we have a deal’ with China’s Xi Ji...,https://nypost.com/2025/10/29/us-news/trump-ch...,148
180,2025-10-30-08-56-55 +0000,cbc,Trump sets new China tariff rate after 'amazin...,https://www.cbc.ca/news/world/us-china-trump-j...,148
230,2025-10-30-02-08-41 +0000,nypost,Trump orders Pentagon to ‘immediately’ restart...,https://nypost.com/2025/10/29/us-news/trump-or...,138
58,2025-10-30-13-46-26 +0000,bbc,Trump hails 'amazing' meeting with China's Xi ...,https://www.bbc.com/news/articles/crl25xl1gjpo...,136


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
20,153,2025-10-30-15-17-58 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...
226,74,2025-10-30-02-28-18 +0000,nypost,Trump says ‘we have a deal’ with China’s Xi Ji...,https://nypost.com/2025/10/29/us-news/trump-ch...
404,72,2025-10-29-18-05-17 +0000,wapo,Report: Government shutdown will cost the econ...,https://www.washingtonpost.com/business/2025/1...
125,72,2025-10-30-10-52-12 +0000,nypost,Husband called wife as he was being crushed to...,https://nypost.com/2025/10/30/us-news/texas-fu...
240,63,2025-10-30-01-32-00 +0000,wsj,Federal Reserve Chair Jerome Powell delivered ...,https://www.wsj.com/economy/central-banking/fe...
399,57,2025-10-29-18-09-32 +0000,nypost,Fed cuts interest rates by quarter point for s...,https://nypost.com/2025/10/29/business/fed-cut...
89,57,2025-10-30-12-41-25 +0000,nypost,Jersey City mayor’s race gets nasty among near...,https://nypost.com/2025/10/30/us-news/jersey-c...
341,53,2025-10-29-20-56-14 +0000,wapo,Hurricane Melissa hits Cuba; has killed at lea...,https://www.washingtonpost.com/weather/2025/10...
65,46,2025-10-30-13-31-37 +0000,nypost,Video shows Zohran Mamdani fervently shouting ...,https://nypost.com/2025/10/30/us-news/video-sh...
93,44,2025-10-30-12-33-40 +0000,nyt,French Police Arrest 5 More in Louvre Jewel He...,https://www.nytimes.com/2025/10/30/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
